/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 
QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


/* Q12: Find the facilities with their usage by member, but not guests */


/* Q13: Find the facilities usage by month, but not guests */

In [2]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    query2 = """
       SELECT sub.name, SUM( sub.revenue ) AS revenue
       FROM (
           SELECT b.facid, b.memid, f.name, f.guestcost, f.membercost, COUNT( b.facid ) AS facid_count,
           CASE
           WHEN b.memid =0
           THEN COUNT( b.facid ) * f.guestcost
           ELSE COUNT( b.facid ) * f.membercost
       END AS 'revenue'
       FROM Bookings AS b
       LEFT JOIN Facilities AS f ON b.facid = f.facid
       GROUP BY b.facid, b.memid
       ) AS sub
       GROUP BY sub.facid
       HAVING revenue <=1000;
        """
    
    query3 = """
       SELECT m.surname, m.firstname, m.recommendedby AS recomender_id, r.surname AS recomender_surname, r.firstname AS recomender_firstname
       FROM Members AS m
       LEFT JOIN Members AS r ON m.recommendedby = r.memid
       WHERE m.recommendedby != 0
       ORDER BY r.surname, r.firstname;
        """
    
    query4 = """
       SELECT b.facid, COUNT( b.memid ) AS mem_usage, f.name
       FROM (
           SELECT facid, memid
           FROM Bookings
           WHERE memid !=0
       ) AS b
       LEFT JOIN Facilities AS f ON b.facid = f.facid
       GROUP BY b.facid;
        """
    query5 = """
        SELECT b.months, COUNT(b.memid) AS mem_usage
        FROM (
            SELECT strftime('%m', starttime) AS months, memid
            FROM Bookings
            WHERE memid != 0
        ) AS b
        GROUP BY b.months;
    """
    
    print('\n results of query 1')
    
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
        
    cur.execute(query2)
    
    rows = cur.fetchall()
    
    print('\n results of question 10')
    for row in rows:
        print(row)
    
    cur.execute(query3)
    
    rows = cur.fetchall()
    
    print('\n results of question 11')
    for row in rows:
        print(row)
        
    cur.execute(query4)
    
    rows = cur.fetchall()
    
    print('\n results of question 12')
    for row in rows:
        print(row)
        
    cur.execute(query5)
    
    rows = cur.fetchall()
    
    print('\n results of question 13')
    for row in rows:
        print(row)
        
    
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks

 results of query 1
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)

 results of question 10
('Badminton Court', 604.5)
('Table Tennis', 90)
('Snooker Table', 115)
('Pool Table', 265)

 results of question 11
('GUEST', 'GUEST', '', None, None)
('Smith', 'Darren', '', None, None)
('Smith', 'Tracy', '', None, None)
('Rownam', 'Tim', '', None, None)
('Tracy', 'Burton', '', None, None)
('Farrell', 'Jemima', '', None, None)
('Farrell', 'David', '', None, None)
('Tupperware', 'Hyacinth', '', None, None)
('Smith', 'Darren', '', None, None)
('Sarwin', 'Ramnaresh', '15', 'Bader', 'Florence')
('Coplin', 'Joan', '16', 'Baker', 'Timothy')
('Genting', 'Matthew', '5', 'Butt